In [1]:
from music21 import *
import glob
from tqdm import tqdm
import numpy as np
import random
from tensorflow.keras.layers import LSTM,Dense,Input,Dropout
from tensorflow.keras.models import Sequential,Model,load_model
from sklearn.model_selection import train_test_split

2022-12-02 20:14:07.954469: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-02 20:14:13.360087: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-02 20:14:13.360138: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-02 20:14:13.808729: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-02 20:14:28.618181: W tensorflow/stream_executor/platform/de

In [15]:
def read_files(file):
    notes=[]
    notes_to_parse=None
    midi=converter.parse(file)
    instrmt=instrument.partitionByInstrument(midi)
    for part in instrmt.parts:
        notes_to_parse=part.recurse()
        for element in notes_to_parse:
            if type(element)==note.Note:
                notes.append(str(element.pitch))
            elif type(element)==chord.Chord:
                notes.append('.'.join(str(n) for n in element.normalOrder))
    return notes
file_path=["chp"]
all_files=glob.glob(file_path[0]+'/*.mid',recursive=True)
notes_array = np.array([read_files(i) for i in tqdm(all_files,position=0,leave=True)],dtype=object)
notes_array

 26%|███████████                               | 36/137 [00:22<01:17,  1.30it/s]/home/civic/anaconda3/lib/python3.9/site-packages/music21/midi/translate.py:863: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=1, channel=None, data=b'NG\xdd\xd5\xccI \xd0A\xccN BA\xcc \xd0ANG Y\xcaU'>; getting generic Instrument
  warnings.warn(
 41%|█████████████████▏                        | 56/137 [00:41<00:33,  2.41it/s]/home/civic/anaconda3/lib/python3.9/site-packages/music21/midi/translate.py:863: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=1, channel=None, data=b'\xd0\xdd\xccNG XA EM NHE\xec'>; getting generic Instrument
  warnings.warn(
/home/civic/anaconda3/lib/python3.9/site-packages/music21/midi/translate.py:863: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=2, channel=None, data=b'\xd0\xdd\xccNG XA EM NHE\xec'>; gettin

array([list(['2.4', '9.11', '11.2.4', '4.9', '9.11.2', '11.2.4', '2.4.9', '4.9', '4.9', '11.4', '4.9', '4.9', '4.9', '11.4', '4.9', '2.6.9', '2.6.9', '2.6.9', 'D3', '9.1.4', '9.1.4', '2.6.9', '2.6.9', '4.8.11', '4.8.11', '8.11.2.4', '4.8.11', '4.8.11', '4.8.11', '4.8.11', '4.8.11', 'E3', 'G#3', '4.8.11', '9.2', '2.6.9', '2.6.9', '2.6.9', '2.6.9', 'A2', '9.1.4', '9.1.4', 'E3', 'E3', '9.1.4', 'E3', '4.8.11', '4.8.11', 'E3', '4.8.11', '4.8.11', 'A2', '9.1.4', 'E3', '11.4', 'E3', '11.4', 'E3', 'G3', '1.4.7', 'A2', 'G3', '1.4.7', 'D3', 'A3', '2.6.9', '2.6.9', '2.6.9', '2.6.9', 'A2', '9.1.4', '9.1.4', '9.1.4', 'E3', '9.1.4', 'G2', '7.11.2', '7.11.2', 'D3', 'A3', '2.6.9', 'A3', '4.9', 'D4', 'E3', '4.8.11', 'B3', 'E4', '4.8.11', 'B3', 'E4', '6.11', 'B3', '6.11', 'B3', '6.11', 'B3', '8.11.2.4', '11.4', '11.4', '11.4', '11.4', '11.4', '11.4', '4.9', '4.9', '11.4', '4.9', '4.9', '4.9', '11.4', '4.9', '2.6.9', '2.6.9', '2.6.9', 'D3', '9.1.4', '9.1.4', '2.6.9', '2.6.9', '4.8.11', '4.8.11', '8.11.2.

In [16]:
notess = sum(notes_array,[])
unique_notes = list(set(notess))
print("Unique Notes:",len(unique_notes))
freq=dict(map(lambda x: (x,notess.count(x)),unique_notes))
freq

Unique Notes: 514


{'9.1': 254,
 '4.7.10.0': 66,
 '7.9.10.0.2.4': 1,
 '11.0.2.4.5.7.8': 4,
 'F#7': 2,
 'C#6': 307,
 '0.4.8': 8,
 '6.7.11': 8,
 '2.5.7.10': 88,
 '0.2.4': 12,
 '0.2.5.8': 2,
 'G6': 85,
 '4.6.7': 2,
 '9.10.0.2': 1,
 '5.6': 24,
 '11.0.1': 2,
 '6.8': 44,
 '1.4.8': 270,
 '3.5.10': 11,
 '9.10.0.2.4.5': 5,
 '9.10.2.5': 55,
 '8.10.1.3': 24,
 'D3': 1282,
 '0.2.7': 52,
 '11.1': 53,
 'E-7': 10,
 'C2': 524,
 '10.0.2.5.7': 12,
 '9.0.3': 20,
 '8.10.1': 15,
 '3.7.11': 33,
 '4.6.9.0': 34,
 'B1': 256,
 '0.4.6': 32,
 '0.2.4.7': 9,
 '6.7.8.10': 1,
 '9.0.1': 2,
 '10.0.1.5': 2,
 '2.4.5.7.9.11': 1,
 '6': 201,
 '1.3.5.8': 13,
 '8.9.1.3': 24,
 '2.4.5.9': 2,
 '2.6': 518,
 '5.6.9': 2,
 '6.9.11': 37,
 '2.5.9': 737,
 '7.10.11': 1,
 '8': 175,
 '10.0.2.4.5.7': 1,
 '8.9.1.4': 3,
 '11.1.2.4': 1,
 '6.8.9.1': 1,
 '11.1.3': 16,
 '10.0.4': 39,
 '0.1.3': 1,
 '7.11.2': 1344,
 '5.7.9.11.0': 3,
 '10.1': 195,
 'G#4': 956,
 '11.3.5': 6,
 '0.2.4.5': 2,
 '7.11.0.2': 3,
 '6.8.10.0.1': 1,
 'F5': 530,
 '4.5.7.8.11': 2,
 '10': 348,
 '7.

In [17]:
THRESHOLD_FREQUENCY=20
freq_notes=dict(filter(lambda x:x[1]>=THRESHOLD_FREQUENCY,freq.items()))
new_notes=[[i for i in j if i in freq_notes] for j in notes_array]

In [18]:
ind2note=dict(enumerate(freq_notes))
note2ind=dict(map(reversed,ind2note.items()))
note2ind

{'9.1': 0,
 '4.7.10.0': 1,
 'C#6': 2,
 '2.5.7.10': 3,
 'G6': 4,
 '5.6': 5,
 '6.8': 6,
 '1.4.8': 7,
 '9.10.2.5': 8,
 '8.10.1.3': 9,
 'D3': 10,
 '0.2.7': 11,
 '11.1': 12,
 'C2': 13,
 '9.0.3': 14,
 '3.7.11': 15,
 '4.6.9.0': 16,
 'B1': 17,
 '0.4.6': 18,
 '6': 19,
 '8.9.1.3': 20,
 '2.6': 21,
 '6.9.11': 22,
 '2.5.9': 23,
 '8': 24,
 '10.0.4': 25,
 '7.11.2': 26,
 '10.1': 27,
 'G#4': 28,
 'F5': 29,
 '10': 30,
 '10.3': 31,
 '10.1.5': 32,
 'C#5': 33,
 '8.10.2': 34,
 '8.11.3': 35,
 '4.7.11': 36,
 '1.4.7': 37,
 '1.3.6': 38,
 '4.8': 39,
 '6.9': 40,
 'C#4': 41,
 '0.4.7': 42,
 'G#1': 43,
 'C#2': 44,
 'F#2': 45,
 '10.2.3': 46,
 'F#4': 47,
 '5.10': 48,
 'A6': 49,
 '1.3': 50,
 '11.0.4.7': 51,
 '3.7': 52,
 'C6': 53,
 '2.5.8': 54,
 '9.0': 55,
 '6.9.0.2': 56,
 '4.5.9.0': 57,
 '2.6.9': 58,
 '3.6.9.11': 59,
 'B-6': 60,
 'B5': 61,
 '4.6': 62,
 '0.2': 63,
 'F#1': 64,
 '3.6': 65,
 '3.4.8.11': 66,
 '7.9.11.2': 67,
 '1.3.7': 68,
 'B6': 69,
 '9.11': 70,
 '7.9': 71,
 '5.7': 72,
 '8.9': 73,
 'F2': 74,
 'B-1': 75,
 'G

In [19]:
timesteps=50
x=[] ; y=[]

for i in new_notes:
    for j in range(0,len(i)-timesteps):
        inp=i[j:j+timesteps] ; out=i[j+timesteps]
        x.append(list(map(lambda x:note2ind[x],inp)))
        y.append(note2ind[out])

x_new=np.array(x)
y_new=np.array(y)

In [20]:
x_new = np.reshape(x_new,(len(x_new),timesteps,1))
y_new = np.reshape(y_new,(-1,1))
x_train,x_test,y_train,y_test = train_test_split(x_new,y_new,test_size=0.2,random_state=42)

In [21]:
model = Sequential()
model.add(LSTM(256,return_sequences=True,input_shape=(x_new.shape[1],x_new.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(256,activation='relu'))
model.add(Dense(len(note2ind),activation='softmax'))




Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 50, 256)           264192    
                                                                 
 dropout_2 (Dropout)         (None, 50, 256)           0         
                                                                 
 lstm_3 (LSTM)               (None, 256)               525312    
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dense_3 (Dense)             (None, 260)               66820     
                                                                 
Total params: 922,116
Trainable params: 922,116
Non-tr

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
model.fit(
 x_train,y_train,
 batch_size=72,epochs=100,
 validation_data=(x_test,y_test))


Epoch 1/30
927/927 [==============================] - 172s 184ms/step - loss: 4.5833 - accuracy: 0.0623 - val_loss: 4.2723 - val_accuracy: 0.1013
Epoch 2/30
927/927 [==============================] - 165s 178ms/step - loss: 4.0931 - accuracy: 0.1287 - val_loss: 3.9172 - val_accuracy: 0.1555
Epoch 3/30
927/927 [==============================] - 163s 175ms/step - loss: 3.7507 - accuracy: 0.1760 - val_loss: 3.6363 - val_accuracy: 0.1985
Epoch 4/30
927/927 [==============================] - 199s 215ms/step - loss: 3.5002 - accuracy: 0.2085 - val_loss: 3.4499 - val_accuracy: 0.2219
Epoch 5/30
927/927 [==============================] - 194s 209ms/step - loss: 3.2988 - accuracy: 0.2369 - val_loss: 3.3128 - val_accuracy: 0.2433
Epoch 6/30
927/927 [==============================] - 182s 197ms/step - loss: 3.1272 - accuracy: 0.2602 - val_loss: 3.1939 - val_accuracy: 0.2600
Epoch 7/30
927/927 [==============================] - 175s 189ms/step - loss: 2.9860 - accuracy: 0.2818 - val_loss: 3.1337 -

In [ ]:
model.save("s2s")

In [ ]:
model = load_model("s2s")
index = np.random.randint(0,len(x_test)-1)
music_pattern = x_test[index]
out_pred=[] #it will store predicted notes
for i in range(400):
    music_pattern = music_pattern.reshape(1,len(music_pattern),1)
    pred_index = np.argmax(model.predict(music_pattern))
    out_pred.append(ind2note[pred_index])
    music_pattern = np.append(music_pattern,pred_index)
    music_pattern = music_pattern[1:]

In [ ]:
all_files1=glob.glob('/*.mid',recursive=True)
notes_array = np.array([read_files(i) for i in tqdm(all_files1,position=0,leave=True)],dtype=object)
notes_array

In [ ]:
output_notes = []
for offset,pattern in enumerate(out_pred):
    if ('.' in pattern) or pattern.isdigit():
         notes_in_chord = pattern.split('.')
         notes = []
         for current_note in notes_in_chord:
            i_curr_note=int(current_note)
            new_note = note.Note(i_curr_note)
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='pred_music.mid')